## Imports

In [1]:
# Canonical
import pandas as pd

# HTTP
import requests

# File utilities
import gzip
import shutil
from pathlib import Path
import csv

# Math
import math

# Download raw data

In [2]:
raw_data_dir = Path("../data/raw")
raw_data_dir.mkdir(parents=True, exist_ok=True)

if any(raw_data_dir.iterdir()):
    pass

else:

    url = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"
    gz_file_path = raw_data_dir / "openfoodfacts_products.csv.gz"
    csv_file_path = raw_data_dir / "openfoodfacts_products.csv"

    print("Downloading OpenFoodFacts CSV file...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(gz_file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Download complete. Extracting...")

    with gzip.open(gz_file_path, "rb") as f_in:
        with open(csv_file_path, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

    gz_file_path.unlink()

    print(f"Extraction complete: {csv_file_path}")

## Flag to avoid preprocessing twice

In [3]:
processed_data_dir = Path("../data/processed")
processed_file = processed_data_dir / "openfoodfacts_processed.csv"

processed_data_dir.mkdir(parents=True, exist_ok=True)

if any(processed_data_dir.iterdir()):
    df = pd.read_csv(processed_file)
    preprocessing = False

else:
    preprocessing = True

# Exploration

## Explore headings

In [4]:
if preprocessing:

    csv_file_path = raw_data_dir / "openfoodfacts_products.csv"

    with open(csv_file_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        print(next(reader))

['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 'last_updated_t', 'last_updated_datetime', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'packaging_en', 'packaging_text', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'origins_en', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'p

## Columns worth keeping

In [5]:
if preprocessing:

    columns = ['url', 'product_name', 'abbreviated_product_name', 'generic_name', 'quantity',
               'ingredients_text', 'ingredients_tags', 'ingredients_analysis_tags',
               'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en',
               'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_n',
               'additives', 'additives_tags', 'additives_en', 'nutriscore_score',
               'nutriscore_grade', 'food_groups', 'food_groups_tags', 'food_groups_en',
               'product_quantity', 'main_category', 'main_category_en', 'image_url',
               'image_small_url', 'image_ingredients_url', 'image_ingredients_small_url',
               'image_nutrition_url', 'image_nutrition_small_url', 'energy-kj_100g',
               'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g', 'fat_100g',
               'saturated-fat_100g', 'butyric-acid_100g', 'caproic-acid_100g',
               'caprylic-acid_100g', 'capric-acid_100g', 'lauric-acid_100g',
               'myristic-acid_100g', 'palmitic-acid_100g', 'stearic-acid_100g',
               'arachidic-acid_100g', 'behenic-acid_100g', 'lignoceric-acid_100g',
               'cerotic-acid_100g', 'montanic-acid_100g', 'melissic-acid_100g',
               'unsaturated-fat_100g', 'monounsaturated-fat_100g', 'omega-9-fat_100g',
               'polyunsaturated-fat_100g', 'omega-3-fat_100g', 'omega-6-fat_100g',
               'alpha-linolenic-acid_100g', 'eicosapentaenoic-acid_100g',
               'docosahexaenoic-acid_100g', 'linoleic-acid_100g', 'arachidonic-acid_100g',
               'gamma-linolenic-acid_100g', 'dihomo-gamma-linolenic-acid_100g',
               'oleic-acid_100g', 'elaidic-acid_100g', 'gondoic-acid_100g', 'mead-acid_100g',
               'erucic-acid_100g', 'nervonic-acid_100g', 'trans-fat_100g',
               'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'added-sugars_100g',
               'sucrose_100g', 'glucose_100g', 'fructose_100g', 'lactose_100g',
               'maltose_100g', 'maltodextrins_100g', 'starch_100g', 'polyols_100g',
               'erythritol_100g', 'fiber_100g', 'soluble-fiber_100g', 'insoluble-fiber_100g',
               'proteins_100g', 'casein_100g', 'serum-proteins_100g', 'nucleotides_100g',
               'salt_100g', 'added-salt_100g', 'sodium_100g', 'alcohol_100g',
               'vitamin-a_100g', 'beta-carotene_100g', 'vitamin-d_100g', 'vitamin-e_100g',
               'vitamin-k_100g', 'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g',
               'vitamin-pp_100g', 'vitamin-b6_100g', 'vitamin-b9_100g', 'folates_100g',
               'vitamin-b12_100g', 'biotin_100g', 'pantothenic-acid_100g', 'silica_100g',
               'bicarbonate_100g', 'potassium_100g', 'chloride_100g', 'calcium_100g',
               'phosphorus_100g', 'iron_100g', 'magnesium_100g', 'zinc_100g', 'copper_100g',
               'manganese_100g', 'fluoride_100g', 'selenium_100g', 'chromium_100g',
               'molybdenum_100g', 'iodine_100g', 'caffeine_100g', 'taurine_100g', 'ph_100g',
               'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-dried_100g',
               'fruits-vegetables-nuts-estimate_100g',
               'fruits-vegetables-nuts-estimate-from-ingredients_100g',
               'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
               'nutrition-score-fr_100g', 'nutrition-score-uk_100g', 'glycemic-index_100g',
               'choline_100g', 'phylloquinone_100g', 'beta-glucan_100g', 'inositol_100g',
               'carnitine_100g', 'sulphate_100g', 'nitrate_100g', 'acidity_100g']

### This reduces the number of columns from 206 to 150, still too many.

In [6]:
if preprocessing:

    chunk_size = 100_000 # Adjust as needed
    frac = 0.315  # Proportion of non-NaN values required to keep a row

    chunks = []

    total_lines = sum(1 for _ in open(csv_file_path, "r", encoding="utf-8"))
    total_chunks = math.ceil(total_lines / chunk_size)

    for i, chunk in enumerate(pd.read_csv(csv_file_path,
                                          sep="\t", usecols=columns,
                                          dtype=str,  # Read all as strings to prevent dtype errors
                                          chunksize=chunk_size,
                                          on_bad_lines="skip"), start=1):  # Start counting from 1

        print(f"\rProcessing chunk {i}/{total_chunks}", end="", flush=True)

        # Drop rows with < frac non-NaN values
        chunk = chunk.dropna(thresh=int(len(chunk.columns) * frac))

        chunks.append(chunk)

    df = pd.concat(chunks, ignore_index=True)

    print("\n\nFinal DataFrame shape:", df.shape)

Processing chunk 37/37

Final DataFrame shape: (9250, 150)


### This reduces our df to the 10,000 most complete rows.

In [7]:
if preprocessing:

    display(df.describe())

url product_name  \
count                                                9250         9202   
unique                                               9242         8097   
top     http://world-en.openfoodfacts.org/product/3250...       Cereal   
freq                                                    2          112   

       abbreviated_product_name  \
count                       460   
unique                      445   
top              Croissance bio   
freq                          3   

                                             generic_name quantity  \
count                                                2664     5635   
unique                                               2268     1965   
top     Matière grasse à tartiner et à cuire allégée (...    500 g   
freq                                                   17      151   

                                         ingredients_text  \
count                                                9182   
unique                                               8122   
top     Sugar, corn flour, wheat flour, whole grain oa...   
freq                                                   16   

                                         ingredients_tags  \
count                                                9182   
unique                                               7648   
top     en:semolina,en:cereal,en:wheat,en:durum-wheat-...   
freq                                                   19   

                                ingredients_analysis_tags  allergens  \
count                                                9186       5509   
unique                                                 34        385   
top     en:palm-oil-free,en:vegan-status-unknown,en:ve...  en:gluten   
freq                                                 2058        990   

       allergens_en  ... nutrition-score-uk_100g glycemic-index_100g  \
count             0  ...                       0                   0   
unique            0  ...                       0                   0   
top             NaN  ...                     NaN                 NaN   
freq            NaN  ...                     NaN                 NaN   

       choline_100g phylloquinone_100g beta-glucan_100g inositol_100g  \
count           153                709                5            89   
unique           86                141                5            53   
top           0.022          0.0000083              3.6         0.029   
freq             10                 73                1             5   

       carnitine_100g sulphate_100g nitrate_100g acidity_100g  
count              70             2            1            0  
unique             28             2            1            0  
top             0.015       0.00053      0.00031          NaN  
freq               16             1            1          NaN  

[4 rows x 150 columns]